# Introduction to relational databases


In [4]:
from sqlalchemy import create_engine

In [ ]:
# Query the right table in information_schema
SELECT table_name 
FROM information_schema.tables
#Specify the correct table_schema value
WHERE table_schema = 'public';

# Query the right table in information_schema to get columns
SELECT column_name, data_type 
FROM information_schema.columns 
WHERE table_name = 'university_professors' AND table_schema = 'public';

# Query the first five rows of our table
SELECT * 
FROM university_professors 
LIMIT 5;

# Tables: At the core of every database


In [ ]:
# syntax for creating tables
CREATE TABLE table_name (
 column_a data_type,
 column_b data_type,
 column_c data_type
);

In [ ]:
# Create a table for the professors entity type
CREATE TABLE professors (
 firstname text,
 lastname text
);

# Print the contents of this table
SELECT * 
FROM professors

#  Create a table for the universities entity type
CREATE TABLE universities (
 university_shortname text,
 university text,
 university_city text
);

# Print the contents of this table
SELECT * 
FROM universities

In [ ]:
# adding table column
ALTER TABLE table_name
ADD COLUMN column_name data_type;

# Add the university_shortname column
ALTER TABLE professors
ADD COLUMN university_shortname text;

# Print the contents of this table
SELECT * 
FROM professors

# Update your database as the structure changes


In [ ]:
# RENAME columns
ALTER TABLE table_name
RENAME COLUMN old_name TO new_name;

# DELETE columns
ALTER TABLE table_name
DROP COLUMN column_name

In [ ]:
# Rename the organisation column
ALTER TABLE affiliations
RENAME COLUMN organisation TO organization;

# Delete the university_shortname column
ALTER TABLE affiliations
DROP COLUMN university_shortname

In [ ]:
INSERT INTO new_table 
SELECT DISTINCT columns 
FROM old_table;

SELECT DISTINCT column_name1, column_name2, ... 
FROM table_a;

INSERT INTO table_b ...;

In [ ]:
## Insert unique professors into the new table
INSERT INTO professors 
SELECT DISTINCT firstname, lastname, university_shortname 
FROM university_professors;

## Doublecheck the contents of professors
SELECT * 
FROM professors;

#Insert unique affiliations into the new table
INSERT INTO affiliations 
SELECT DISTINCT firstname, lastname, function, organization 
FROM university_professors;

# Doublecheck the contents of affiliations
SELECT * 
FROM affiliations;

In [ ]:
# Delete the university_professors table
DROP TABLE university_professors

# Better data quality with constraints


In [ ]:
CREATE TABLE transactions (
 transaction_date date, 
 amount integer,
 fee text
);

# date is in yyyy-mm-dd

In [ ]:
# Let's add a record to the table
INSERT INTO transactions (transaction_date, amount, fee) 
VALUES ('2018-09-24', 5454, '30');

-- Doublecheck the contents
SELECT *
FROM transactions;

In [ ]:
# to write function
SELECT CAST(some_column AS integer)
FROM table;

# Calculate the net amount as amount + fee
SELECT transaction_date, amount + CAST(fee AS integer) AS net_amount 
FROM transactions;

# Working with data types


In [ ]:
# HOW to CHANGE DATA TYPE
ALTER TABLE table_name
ALTER COLUMN column_name
TYPE varchar(10)

In [ ]:
# Select the university_shortname column
SELECT DISTINCT (university_shortname)
FROM professors;

#Specify the correct fixed-length character type
ALTER TABLE professors
ALTER COLUMN university_shortname
TYPE char(3);

#Change the type of firstname
ALTER TABLE professors
ALTER COLUMN firstname
TYPE varchar(64);

In [ ]:
ALTER TABLE table_name
ALTER COLUMN column_name
TYPE varchar(x)
USING SUBSTRING(column_name FROM 1 FOR x)

In [ ]:
# Convert the values in firstname to a max. of 16 characters
ALTER TABLE professors 
ALTER COLUMN firstname 
TYPE varchar(16) 
USING SUBSTRING(firstname FROM 1 FOR 16);

# The not-null and unique constraints


In [ ]:
# Disallow NULL values in firstname
ALTER TABLE professors 
ALTER COLUMN firstname SET NOT NULL;

# Disallow NULL values in lastname
ALTER TABLE professors
ALTER COLUMN lastname SET NOT NULL

In [ ]:
CREATE TABLE table_name (
 column_name UNIQUE
);

ALTER TABLE table_name
ADD CONSTRAINT some_name UNIQUE(column_name);

In [ ]:
# Make universities.university_shortname unique
ALTER TABLE universities
ADD CONSTRAINT university_shortname_unq UNIQUE(university_shortname);

# Make organizations.organization unique
ALTER TABLE organizations
ADD CONSTRAINT organization_unq UNIQUE(organization)

# Keys and SuperKeys

In [ ]:
# how to find if a certain column has distinct values
SELECT COUNT(DISTINCT(column_a, column_b, ...))
FROM table;

# Count the number of rows in universities
SELECT COUNT(*) 
FROM universities;

# Count the number of distinct values in the university_city column
SELECT COUNT(DISTINCT(university_city)) 
FROM universities;

In [ ]:
# super key
# Try out different combinations
SELECT COUNT(DISTINCT(firstname, lastname)) 
FROM professors;

# Primary keys

In [ ]:
ALTER TABLE table_name
ADD CONSTRAINT some_name PRIMARY KEY (column_name)

# Rename the organization column to id
ALTER TABLE organizations
RENAME COLUMN organization TO id;

# Make id a primary key
ALTER TABLE organizations
ADD CONSTRAINT organization_pk PRIMARY KEY (id);

# Rename the university_shortname column to id
ALTER TABLE universities
RENAME COLUMN university_shortname TO id;

# Make id a primary key
ALTER TABLE universities
ADD CONSTRAINT university_pk PRIMARY KEY (id);

# Surrogate keys


In [ ]:
# Add the new column to the table
ALTER TABLE professors 
ADD COLUMN id serial;

# Make id a primary key
ALTER TABLE professors 
ADD CONSTRAINT professors_pkey PRIMARY KEY (id);

# Have a look at the first 10 rows of professors
SELECT *
FROM professors
LIMIT 10

In [ ]:
CREATE TABLE cars (
 make varchar(64) NOT NULL,
 model varchar(64) NOT NULL,
 mpg integer NOT NULL
)

#-- Count the number of distinct rows with columns make, model
SELECT COUNT(DISTINCT(make, model)) 
FROM cars;

# Add the id column
ALTER TABLE cars
ADD COLUMN id varchar(128);

# Update id with make + model
UPDATE cars
SET id = CONCAT(make, model);

# Make id a primary key
ALTER TABLE cars
ADD CONSTRAINT id_pk primary key(id);

# Have a look at the table
SELECT * FROM cars;

In [ ]:
# Create the table
CREATE TABLE students (
  last_name varchar(128) NOT NULL,
  ssn integer PRIMARY KEY,
  phone_no char(12)
);

# Model 1:N relationships with foreign keys


In [ ]:
ALTER TABLE a 
ADD CONSTRAINT a_fkey FOREIGN KEY (b_id) REFERENCES b (id);

# Rename the university_shortname column
ALTER TABLE professors
RENAME COLUMN university_shortname TO university_id;

# -- Add a foreign key on professors referencing universities
ALTER TABLE professors 
ADD CONSTRAINT professors_fkey FOREIGN KEY (university_id) REFERENCES universities (id);

In [ ]:
# adding foreign allows to pick up mistakes
#Try to insert a new professor
INSERT INTO professors (firstname, lastname, university_id)
VALUES ('Albert', 'Einstein', 'UZH');

In [ ]:
# joining data
SELECT ...
FROM table_a
JOIN table_b
ON ...
WHERE ...

# Select all professors working for universities in the city of Zurich
SELECT professors.lastname, universities.id, universities.university_city
FROM professors
JOIN universities
ON professors.university_id = universities.id
WHERE universities.university_city = 'Zurich';

# Model more complex relationships


In [ ]:
# Add a professor_id column
ALTER TABLE affiliations
ADD COLUMN professor_id integer REFERENCES professors (id);

# Rename the organization column to organization_id
ALTER TABLE affiliations
RENAME organization TO organization_id;

# Add a foreign key on organization_id
ALTER TABLE affiliations
ADD CONSTRAINT affiliations_organization_fkey FOREIGN KEY (organization_id) REFERENCES organizations (id);

In [ ]:
UPDATE table_a
SET column_to_update = table_b.column_to_update_from
FROM table_b
WHERE condition1 AND condition2 AND ...;

# Update professor_id to professors.id where firstname, lastname correspond to rows in professors
UPDATE affiliations
SET professor_id = professors.id
FROM professors
WHERE affiliations.firstname = professors.firstname AND affiliations.lastname = professors.lastname;

# Have a look at the 10 first rows of affiliations again
SELECT * FROM affiliations LIMIT 10;

In [ ]:
#  Drop the firstname column
ALTER TABLE affiliations
DROP COLUMN firstname;

# Drop the lastname column
ALTER TABLE affiliations
DROP COLUMN lastname;

# Referential integrity


In [ ]:
# Identify the correct constraint name
SELECT constraint_name, table_name, constraint_type
FROM information_schema.table_constraints
WHERE constraint_type = 'FOREIGN KEY';

# Drop the right foreign key constraint
ALTER TABLE affiliations
DROP CONSTRAINT affiliations_organization_id_fkey;

# Add a new foreign key constraint from affiliations to organizations which cascades deletion
ALTER TABLE affiliations
ADD CONSTRAINT affiliations_organization_id_fkey FOREIGN KEY (organization_id) REFERENCES organizations (id) ON DELETE CASCADE;

# Delete an organization 
DELETE FROM organizations 
WHERE id = 'CUREM';

# Check that no more affiliations with this organization exist
SELECT * FROM affiliations
WHERE organization_id = 'CUREM';

# Roundup


In [ ]:
SELECT table_a.column1, table_a.column2, table_b.column1, ... 
FROM table_a
JOIN table_b 
ON table_a.column = table_b.column

In [ ]:
# Count the total number of affiliations per university
SELECT COUNT(*), professors.university_id 
FROM affiliations
JOIN professors
ON affiliations.professor_id = professors.id
# Group by the university ids of professors
GROUP BY professors.university_id 
ORDER BY count DESC;

In [ ]:
# Filter the table and sort it
SELECT COUNT(*), organizations.organization_sector, 
professors.id, universities.university_city
FROM affiliations
JOIN professors
ON affiliations.professor_id = professors.id
JOIN organizations
ON affiliations.organization_id = organizations.id
JOIN universities
ON professors.university_id = universities.id
WHERE organizations.organization_sector = 'Media & communication'
GROUP BY organizations.organization_sector, 
professors.id, universities.university_city
ORDER BY COUNT DESC;